# Stream Model From AWS S3

In this notebook we will demonstrate how to read tensors using the RunAI Model Streamer from AWS S3 bucket.

## Prerequisites
* A Linux machine
* An AWS S3 bucket
* Credentials to read from the bucket

## Preperation
Set the following varibales with your credentials and bucket:

In [ ]:
s3_bucket = 'your-bucket-name'
s3_location = 'desired/path/in/s3'

s3_access_key = 'your-access-key'
s3_access_secret = 'your-secret-access-key'

Now we will set them as standard AWS environment varibles:

In [ ]:
import os

os.environ['AWS_ACCESS_KEY_ID'] = s3_access_key
os.environ['AWS_SECRET_ACCESS_KEY'] = s3_access_secret

We will start by downloading an example `.safetensors` file. Feel free to use your own.

In [1]:
import subprocess

url = 'https://huggingface.co/vidore/colpali/resolve/main/adapter_model.safetensors?download=true'
local_filename = 'model.safetensors'

wget_command = ['wget', '--content-disposition', url, '-O', local_filename]
subprocess.run(wget_command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

Next thing we would do is upload the model `.safetensors` file to the S3 bucket:

In [ ]:
import boto3

local_file = 'model.safetensors'
s3_upload_location = f"{s3_location}/{local_file}"

s3_client = boto3.client(
    's3',
    aws_access_key_id=s3_access_key,
    aws_secret_access_key=s3_access_secret
)

s3_client.upload_file(local_file, s3_bucket, s3_upload_location)

## Streaming

To load the tensors from the S3 file we need to create `SafetensorsStreamer` instance, perform the request, and start getting the tensors:

In [ ]:
from runai_model_streamer import SafetensorsStreamer

file_path = f"s3://{s3_bucket}/{s3_upload_location}"

def heavy_workload(tensor):
    # Perform heavy computation with the tensor
    # The computation is occured during the reading
    # of the rest of the tensors from the storage
    return

with SafetensorsStreamer() as streamer:
    streamer.stream_file(file_path)
    for name, tensor in streamer.get_tensors():
        heavy_workload(tensor)